# Explore ways to read/write params to/from a file



In [1]:
from see import base_classes 

In [2]:
from see.Segmentors import segmentor
from see.ColorSpace import colorspace
from see.Workflow import workflow
from see.Segment_Fitness import segment_fitness

workflow.addalgos([colorspace, segmentor, segment_fitness])
wf = workflow()

----

# Use Pickle to wrap entire algorithm object

This works nice assuming the algorithm is what we want to write. Dosn't work in the genetic algoirthm since it may just be a list. Although it works fairly well.

In [3]:
import pickle 
def append_algo_pickle(fpop_file, algorithm):
    filehandler = open(fpop_file, 'ab') 
    pickle.dump(algorithm, filehandler)
    
def read_algo_pickle(fpop_file):
    poplist = []
    with (open(fpop_file, "rb")) as openfile:
        while True:
            try:
                poplist.append(pickle.load(openfile))
            except EOFError:
                break
    return poplist
           

In [4]:
!rm pickle_test.pk

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
append_algo_pickle('pickle_test.pk', wf)

In [6]:
pop = read_algo_pickle('pickle_test.pk')
pop

In [7]:
!cat 'pickle_test.pk'

'cat' is not recognized as an internal or external command,
operable program or batch file.


---
This version just saves the list

In [8]:
!rm pickle_test.pk

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
append_algo_pickle('pickle_test.pk', wf.params.tolist())

In [10]:
pop = read_algo_pickle('pickle_test.pk')
pop

 ['RGB', True, 2, 'ColorThreshold', 0.3, 0.5, 0.2, 0.7, 0.3, 0.5, 2, 10]]

---

# Use JSON

Read in as a json file. This also dosn't work as well since we can't append additional values to the list.

In [11]:
import json
def append_algo_json(fpop_file, algorithm):
    with open('json_test.json', 'w', encoding="utf8") as f:
        json.dump(algorithm, f)

def read_algo_json(fpop_file):
    with open(fpop_file, 'r') as f:
        population = json.load(f)
    return population

In [12]:
!rm 'json_test.json'

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
append_algo_json('json_test.json', wf.params)

In [14]:
!cat 'json_test.json'

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
pop = read_algo_json('json_test.json')

----

# Basic Vector

This is a more direct way to write out the list. The nice thing about this format is it is human readable.  Maybe not as flexible but easier to work wiht.

In [16]:
def write_algo_vector(fpop_file, algorithm):
    with open(f"{fpop_file}", 'a') as myfile:
        myfile.write(f'{algorithm.params.tolist().__str__()}\n')
        
def read_algo_vector(fpop_file):
    inlist = []
    with open(f"{fpop_file}",'r') as myfile:
        for line in myfile:
            inlist.append(eval(line))
    return inlist

In [17]:
!rm 'list_test.txt'

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
write_algo_vector('list_test.txt', wf)

In [19]:
!cat List_test.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
params_as_list = read_algo_vector('list_test.txt')
params_as_list

[['RGB', True, 2, 'ColorThreshold', 0.3, 0.5, 0.2, 0.7, 0.3, 0.5, 2, 10]]

----

# Basic Dictionary

Same idea as a list but as a dictionary.  Has the same problem as the basic pickle. 

In [21]:
def write_algo_dict(fpop_file, params):
    with open(f"{fpop_file}", 'a') as myfile:
        myfile.write(f'{dict(params).__str__()}\n')
        
def read_algo_dict(fpop_file):
    inlist = []
    with open(f"{fpop_file}",'r') as myfile:
        for line in myfile:
            inlist.append(eval(line))
    return inlist

In [22]:
!rm 'dict_test.txt'

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [23]:
write_algo_dict('dict_test.txt', wf.params)

In [24]:
!cat dict_test.txt

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [25]:
params_as_dict = read_algo_vector('dict_test.txt')
params_as_dict

[{'colorspace': 'RGB',
  'multichannel': True,
  'channel': 2,
  'algorithm': 'ColorThreshold',
  'alpha1': 0.3,
  'alpha2': 0.5,
  'beta1': 0.2,
  'beta2': 0.7,
  'gamma1': 0.3,
  'gamma2': 0.5,
  'n_segments': 2,
  'max_iter': 10}]

# Testing in SEE


In [26]:
from see import base_classes

In [27]:
import imageio
img = imageio.imread('Image_data/Examples/AA_Chameleon.jpg')
gmask = imageio.imread('Image_data/Examples/AA_Chameleon_GT.png')

mydata = base_classes.pipedata()

In [28]:
from see.Segmentors import segmentor
from see.ColorSpace import colorspace
from see.Workflow import workflow
from see.Segment_Fitness import segment_fitness

workflow.addalgos([colorspace, segmentor, segment_fitness])
wf = workflow()
print(wf)

<class 'see.Workflow.workflow'> parameters: 
	colorspace = RGB
	multichannel = True
	channel = 2
	algorithm = ColorThreshold
	alpha1 = 0.3
	alpha2 = 0.5
	beta1 = 0.2
	beta2 = 0.7
	gamma1 = 0.3
	gamma2 = 0.5
	n_segments = 2
	max_iter = 10



In [29]:
from see import GeneticSearch

In [30]:
my_evolver = GeneticSearch.Evolver(workflow, mydata, pop_size=10)

In [31]:
test = my_evolver.newpopulation()

In [32]:
for ind in test:
    GeneticSearch.write_algo_vector("Dirktest.txt", ind)

Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt
Writing in Dirktest.txt


In [33]:
test2 = GeneticSearch.read_algo_vector("Dirktest.txt")
test3 = my_evolver.copy_pop_list(test2)

Reading in Dirktest.txt


In [34]:
type(test3[0])

deap.creator.Individual

In [35]:
!cat Dirktest.txt


'cat' is not recognized as an internal or external command,
operable program or batch file.


In [36]:
from pathlib import Path

In [37]:
filen = Path('Dirktest.txt')

In [38]:
dir(filen)

['__bytes__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__fspath__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rtruediv__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__truediv__',
 '_accessor',
 '_cached_cparts',
 '_cparts',
 '_drv',
 '_flavour',
 '_format_parsed_parts',
 '_from_parsed_parts',
 '_from_parts',
 '_hash',
 '_init',
 '_make_child',
 '_make_child_relpath',
 '_opener',
 '_parse_args',
 '_parts',
 '_pparts',
 '_raw_open',
 '_root',
 '_str',
 'absolute',
 'anchor',
 'as_posix',
 'as_uri',
 'chmod',
 'cwd',
 'drive',
 'exists',
 'expanduser',
 'glob',
 'group',
 'home',
 'is_absolute',
 'is_block_device',
 'is_char_device',
 'is_dir',
 'is_fifo',
 'is_file',
 'is_mount',
 'is_relative_to',
 'is_re

In [39]:
filen.suffix

'.txt'